In [240]:
import pandas as pd

In [241]:
df = pd.read_json('data/formatted.json', orient='index')
df.shape

(145, 75)

#### First look at our data after scraping

In [270]:
df.head(10)

,power_index,ppp,forex_gold,defense_budget,external_debt,square_land_area,coastline_coverage,shared_borders,waterways,population,...,ports,airports,roadway_coverage,railway_coverage,total_naval_assets,aircraft_carriers,submarines,army_readiness,airforce_readiness,artillery
afghanistan,2.3118,18000000000,5000000000,49525000,145000000,652230,0,5987,1200,38346720,...,2,46,34903,0,0.0,0.0,0.0,0.181538,0.000000,60
albania,1.8466,38000000000,4000000000,192400000,10000000000,28748,362,691,41,3095344,...,4,3,3945,677,19.0,0.0,0.0,0.200000,0.526316,0
algeria,0.3911,470000000000,100000000000,9979200000,6000000000,2381741,998,6734,0,44178884,...,12,149,104000,3973,201.0,0.0,6.0,0.650582,0.650823,707
angola,0.8732,205700000000,16995000000,1059270000,45000000000,1246700,1600,5369,1300,34795287,...,6,102,26000,2852,40.0,0.0,0.0,0.647676,0.599338,653
argentina,0.4243,900000000000,45000000000,3064800000,280000000000,2780400,4989,11968,11000,46245668,...,10,916,281290,36917,47.0,0.0,2.0,0.547634,0.548246,247
armenia,1.9137,40000000000,2200000000,632500000,12000000000,29743,0,1570,0,3000756,...,0,7,7700,780,0.0,0.0,0.0,0.551046,0.500000,183
australia,0.2567,1250900000000,60000000000,52162000000,3115913000000,7741220,25760,0,2000,26141369,...,29,418,873573,33343,43.0,0.0,6.0,0.387980,0.648725,48
austria,1.6543,465100000000,22000000000,3432000000,700000000000,83871,0,2524,0,8913088,...,4,50,137039,5800,0.0,0.0,0.0,0.386861,0.653465,53
azerbaijan,0.9391,140000000000,6700000000,3100000000,20000000000,86600,713,2468,0,10353296,...,1,23,24981,2944,24.0,0.0,4.0,0.650352,0.547297,486
bahrain,1.4511,70000000000,2349000000,1538800000,55000000000,760,161,0,0,1540588,...,2,4,4122,0,48.0,0.0,0.0,0.553034,0.500000,70


In [243]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, afghanistan to zimbabwe
Data columns (total 75 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   pwrIndex                         145 non-null    float64
 1   Purchasing Power Parity:         145 non-null    object 
 2   Foreign Exchange/Gold:           145 non-null    object 
 3   Defense Budget:                  145 non-null    object 
 4   External Debt:                   145 non-null    object 
 5   Square Land Area:                145 non-null    object 
 6   Coastline Coverage:              145 non-null    object 
 7   Shared Borders:                  145 non-null    object 
 8   Waterways (usable):              145 non-null    object 
 9   Total Population:                145 non-null    object 
 10  Available Manpower               145 non-null    object 
 11  Fit-for-Service                  145 non-null    object 
 12  Reaching Mil

#### Missing Data

In [244]:

nulls = df.isnull().sum().sort_values(ascending=False).head(10)
nulls = nulls[nulls > 0]
nulls

Mine Warfare:           32
Patrol Vessels:         32
Submarines:             32
Corvettes:              32
Frigates:               32
Destroyers:             32
Helicopter Carriers:    32
Aircraft Carriers:      32
Total Assets:           32
dtype: int64

Are these the same countries?

In [245]:
cols = nulls.index.tolist()

# find the list of countries whose values are null for each column
countries_without_navies = []
for col in cols:
	countries_with_null = df[df[col].isnull()].index.tolist()
	countries_without_navies.extend(countries_with_null)

countries_without_navies = set(countries_without_navies)
len(countries_without_navies)

32

In [246]:
list(countries_without_navies)[:5]

['mongolia', 'paraguay', 'niger', 'tajikistan', 'afghanistan']

Problem: Navy data is missing for 32 countries, and it's not on the GFP website.
We also have a lot of columns

We drop a few of these navy columns because they are specialized forms of navy ships that do not serve our macro analysis. This also helps reduce the dimensionality of our data

In [247]:
to_drop = ['Mine Warfare:', 'Helicopter Carriers:', 'Destroyers:', 'Frigates:', 'Corvettes:','Patrol Vessels:']
df.drop(to_drop, axis=1, inplace=True)

We replace nulls for Submarines and Aircraft Carriers with zeros and keep them as well as total naval assets as the indicators for navy capability. This will help us identify super powers with such assets

In [248]:
df[["Submarines:", "Aircraft Carriers:"]].value_counts(ascending=True)

Submarines:  Aircraft Carriers:
9.0          1.0                    1
68.0         11.0                   1
35.0         0.0                    1
22.0         0.0                    1
21.0         0.0                    1
19.0         0.0                    1
18.0         2.0                    1
12.0         0.0                    1
11.0         0.0                    1
10.0         2.0                    1
70.0         1.0                    1
78.0         2.0                    1
8.0          2.0                    1
             0.0                    1
7.0          0.0                    1
3.0          0.0                    1
9.0          0.0                    1
1.0          0.0                    2
5.0          0.0                    3
6.0          0.0                    6
4.0          0.0                    7
2.0          0.0                    8
0.0          0.0                   70
Name: count, dtype: int64

In [249]:
df["Submarines:"].fillna(0, inplace=True)
df["Aircraft Carriers:"].fillna(0, inplace=True)
df["Total Assets:"].fillna(0, inplace=True)

Let's see what's left

In [250]:
nulls = df.isnull().sum().sort_values(ascending=False).head(10)
nulls = nulls[nulls > 0]
nulls

Series([], dtype: int64)

#### Fixing Data types

Most of our data consists of strings. let's convert them to numeric types

In [251]:
import numpy as np

df['pwrIndex'] = df['pwrIndex'].astype(np.float64)

def convert_to_int(x):
	if isinstance(x, str):
		x = x.replace(',', '')
		x = x.replace('$', '')
		x = x.replace(' ', '')
		
		# units
		x = x.replace('USD', '')
		x = x.replace('km', '')
		x = x.replace('bbl', '')
		x = x.replace('cu.m', '')
		x = x.replace('mt', '')
		
		# Coastal Coverage and Shared Borders values
		x = x.replace('LANDLOCKED', '0')
		x = x.replace('NONE-ISLAND', '0')
		
		return np.int64(x)
	else:
		return x

for col in df.columns:
	df[col] = df[col].apply(convert_to_int)

In [252]:
df.isna().sum().sum()

0

### We still have a lot of columns

In [253]:
df.shape

(145, 69)

Let's replace the features which represent the number of these forces which are ready and represent that in a ratio for the entire devision. We'll create a readiness ratio for Army and Airforce as navy does not have data on readiness.

In [254]:
army = [
	'Tanks:',
	'Vehicles:',
	'Self-Propelled Artillery:',
	'Towed Artillery:',
	'MLRS (Rocket Artillery):'
]

In [255]:
df.loc['hungary', army + [a + ' Ready' for a in army]]

Tanks:                                176.0
Vehicles:                            7128.0
Self-Propelled Artillery:              10.0
Towed Artillery:                      295.0
MLRS (Rocket Artillery):                0.0
Tanks: Ready                        11440.0
Vehicles: Ready                    463320.0
Self-Propelled Artillery: Ready       650.0
Towed Artillery: Ready              19175.0
MLRS (Rocket Artillery): Ready          0.0
Name: hungary, dtype: float64

In [256]:
for a in army:
	df[a + '_army_readiness_ratio'] = df[a + ' Ready'] / df[a]
	df[a + '_army_readiness_ratio'].fillna(0, inplace=True)

ratio_cols = [a + '_army_readiness_ratio' for a in army]
df["army_readiness"] = df[ratio_cols].mean(axis=1)
df["army_readiness"].head(5)

afghanistan    0.181538
albania        0.200000
algeria        0.650582
angola         0.647676
argentina      0.547634
Name: army_readiness, dtype: float64

In [257]:
df["airforce_readiness"] = df['Aircraft Total: Ready'] / df['Aircraft Total:']
df["airforce_readiness"].fillna(0, inplace=True)
df["airforce_readiness"].sort_values(ascending=False).head(10)

luxembourg        1.000000
sweden            0.848780
spain             0.800774
taiwan            0.800543
israel            0.800333
south-korea       0.800250
japan             0.800138
turkey            0.800000
united-kingdom    0.799397
switzerland       0.797297
Name: airforce_readiness, dtype: float64

In [258]:
df = df.drop(df.filter(regex='Ready').columns, axis=1)
df = df.drop(df.filter(regex='army_readiness_ratio').columns, axis=1)

In [259]:
df.isna().sum().sum()

0

In [260]:
 df.shape

(145, 57)

Let's merge two types of artillery into one

In [261]:
df['artillery'] = df['Self-Propelled Artillery:'] + df['Towed Artillery:']
df.drop(['Self-Propelled Artillery:', 'Towed Artillery:'], axis=1, inplace=True)
df.shape

(145, 56)

### Let's fix column names

In [262]:
df.columns

Index(['pwrIndex', 'Purchasing Power Parity:', 'Foreign Exchange/Gold:',
       'Defense Budget:', 'External Debt:', 'Square Land Area:',
       'Coastline Coverage:', 'Shared Borders:', 'Waterways (usable):',
       'Total Population:', 'Available Manpower', 'Fit-for-Service',
       'Reaching Mil Age Annually', 'Tot Mil. Personnel (est.)',
       'Active Personnel', 'Reserve Personnel', 'Paramilitary',
       'Air Force Personnel*', 'Army Personnel*', 'Navy Personnel*',
       'Aircraft Total:', 'Fighters:', 'Attack Types:',
       'Transports (Fixed-Wing):', 'Trainers:', 'Special-Mission:',
       'Tanker Fleet:', 'Helicopters:', 'Attack Helicopters:', 'Tanks:',
       'Vehicles:', 'MLRS (Rocket Artillery):', 'Oil Production:',
       'Oil Consumption:', 'Oil Deficit:', 'Oil Proven Reserves:',
       'Natural Gas Production:', 'Natural Gas Consumption:',
       'Natural Gas Deficit:', 'Nat.Gas Proven Rez:', 'Coal Production:',
       'Coal Consumption:', 'Coal Deficit:', 'Coal Prove

In [263]:
columns = {
	'Purchasing Power Parity:': 'ppp',
	'Foreign Exchange/Gold:': 'forex_gold',
	'Waterways (usable):': 'waterways',
	'Total Population:': 'population',
	'Tot Mil. Personnel (est.)': 'total_personnel',
	'Transports (Fixed-Wing):': 'transports',
	'MLRS (Rocket Artillery):': 'mlrs',
	'Ports / Trade Terminals:': 'ports',
	'pwrIndex': 'power_index',
	'Total Assets:': 'total_naval_assets',
}
df.rename(columns=columns, inplace=True)

Remaining Column Titles

In [264]:
columns = list(df.columns)
rename_dict = {}

for title in columns:
	new_title = title.lower()
	new_title = new_title.replace(':', '')

	new_title = new_title.replace('-', '_')
	new_title = new_title.replace('nat.gas', 'nat_gas')
	new_title = new_title.replace('natural', 'nat')
	new_title = new_title.replace('production', 'prod')
	new_title = new_title.replace('consumption', 'cons')
	new_title = new_title.replace('proven', '')
	new_title = new_title.replace('reserves', 'res')
	new_title = new_title.replace('rez', 'res')
	new_title = new_title.replace(' ', '_')
	new_title = new_title.replace('__', '_')
	new_title = new_title.replace('*', '')

	rename_dict[title] = new_title

df.rename(columns=rename_dict, inplace=True)
df.columns

Index(['power_index', 'ppp', 'forex_gold', 'defense_budget', 'external_debt',
       'square_land_area', 'coastline_coverage', 'shared_borders', 'waterways',
       'population', 'available_manpower', 'fit_for_service',
       'reaching_mil_age_annually', 'total_personnel', 'active_personnel',
       'reserve_personnel', 'paramilitary', 'air_force_personnel',
       'army_personnel', 'navy_personnel', 'aircraft_total', 'fighters',
       'attack_types', 'transports', 'trainers', 'special_mission',
       'tanker_fleet', 'helicopters', 'attack_helicopters', 'tanks',
       'vehicles', 'mlrs', 'oil_prod', 'oil_cons', 'oil_deficit', 'oil_res',
       'nat_gas_prod', 'nat_gas_cons', 'nat_gas_deficit', 'nat_gas_res',
       'coal_prod', 'coal_cons', 'coal_deficit', 'coal_res', 'labor_force',
       'merchant_marine_fleet', 'ports', 'airports', 'roadway_coverage',
       'railway_coverage', 'total_naval_assets', 'aircraft_carriers',
       'submarines', 'army_readiness', 'airforce_readiness'

#### Let's get the latitude and longitude for each country to visualize them!

In [265]:
countries = df.index.tolist()

In [267]:
import time 
from geopy.geocoders import Nominatim

latitude = pd.Series()
longitude = pd.Series()

def get_country_coordinates(row):
    country_name = row.name.replace('-', ' ')
    geolocator = Nominatim(
        user_agent="military_data_analysis")
    location = geolocator.geocode(country_name, featuretype="country")

    time.sleep(0.2)
    if location:
        latitude[row.name] = location.latitude
        longitude[row.name] = location.longitude
    else:
        return None

In [268]:
df.apply(get_country_coordinates, axis=1)
df['lat'] = latitude
df['long'] = longitude

In [ ]:
# round all floats to 3 decimal places, except power index, and lat/long
float_cols = df.select_dtypes(include=['float64']).columns.tolist()
float_cols.remove('power_index')
float_cols.remove('lat')
float_cols.remove('long')

for col in float_cols:
	df[col] = df[col].round(3)

In [ ]:
df.to_csv('data/clean.csv')